In [ ]:
import pandas as pd 
import requests
import datetime
from urllib.parse import urlencode
import json
import sqlite3
import sqlalchemy


In [ ]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_token_header(self):
        return {
            "Content-Type": "application/x-www-form-urlencoded"
        }

    def get_token_data(self):
        return {
            "grant_type": "client_credentials",
            "client_id": self.client_id,
            "client_secret": self.client_secret
        }

    def perform_authentication(self):
        response = requests.post(self.token_url,
                                 data=self.get_token_data(),
                                 headers=self.get_token_header())

        if response.status_code not in range(200, 299):
            raise Exception("could not authenticate client")
        token_response_data = response.json()
        now = datetime.datetime.now()
        self.access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return expires_in, expires, now

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()

        if expires < now:
            self.perform_authentication()
            return self.get_access_token()
        elif token is None:
            self.perform_authentication()
            return self.get_access_token()
        return token

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers

    def get_resource(self, _id, resource_type):
        endpoint = f"https://api.spotify.com/v1/{resource_type}/{_id}"
        headers = self.get_resource_header()
        response = requests.get(endpoint, headers=headers)
        if response.status_code not in (200, 299):
            return response.status_code
        return response.json()

    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')

    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

    def search(self, query, search_type):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({
            "q": query,
            "type": search_type.lower(),
            "market": "NG",
            "limit":50
        })
        lookup_url = f"{endpoint}?{data}"
        spotify_response = requests.get(lookup_url, headers=headers)
        if spotify_response.status_code in range(200, 299):
            return spotify_response.json()
        return spotify_response.status_code

      # def get_recommendations(self,)

In [ ]:
spotify_client_id = "3ed8b4ef53864cddbda665838a95d614"
spotify_client_secret = "e27a6cc4fa19438b8139d25c22116cf7"

spotify = SpotifyAPI(spotify_client_id, spotify_client_secret)

In [ ]:
# build our data Frame 


def create_dataframe(genre, search_type='track'):

  music = spotify.search(genre,search_type)
  artist_music = music['tracks']['items']

  df_name = pd.DataFrame(columns=['name','followers','song_genres','album_label','album_name','album_popularity','album_release_date','album_total_tracks'])

  for items in artist_music:
    album_id = items['album']['id']
    artist_id = items['album']['artists'][0]['id']

    artist = spotify.get_artist(artist_id)
    album = spotify.get_album(album_id)

  
    artist_name = artist['name']
    artist_followers = artist['followers']['total']
    artist_genres = artist['genres']
    artist_genres = ", ".join([str(item) for item in artist_genres])
    album_label = album['label']
    album_name = album['name']
    album_popularity = album['popularity']
    album_release_date = album['release_date']
    album_total_tracks = album['total_tracks']


    df_name = df_name.append({
        'name': artist_name,
        'followers': artist_followers,
        'song_genres': artist_genres,
        'album_label': album_label,
        'album_name': album_name,
        'album_popularity':album_popularity,
        'album_release_date':album_release_date,
        'album_total_tracks': album_total_tracks
    }, ignore_index=True) 

  return df_name



In [ ]:
df_azontobeats = create_dataframe('genre:azontobeats')
df_afropop = create_dataframe('genre:afropop')
df_nigerian_pop = create_dataframe('genre:nigerian pop')

In [ ]:
df_afropop.head(10)

,name,followers,song_genres,album_label,album_name,album_popularity,album_release_date,album_total_tracks
0,Kizz Daniel,734191,"afropop, azontobeats, nigerian pop",FlyBoy INC / EMPIRE,Buga (Lo Lo Lo),59,2022-05-03,1
1,Sarz,116912,"afro dancehall, afropop, nigerian pop",Metallic Music/1789,Monalisa,59,2022-05-19,1
2,WizKid,2409832,"afro dancehall, afropop, azonto, nigerian hip ...",Starboy/RCA Records,Made In Lagos,65,2020-10-29,14
3,Oxlade,193953,"afropop, nigerian pop",Epic,KU LO SA - A COLORS SHOW,59,2022-06-10,1
4,Olamide,877140,"afro dancehall, afropop, azonto, nigerian hip ...",YBNL Nation / EMPIRE,Carpe Diem,60,2020-10-07,12
5,L.A.X,132565,"afropop, azontobeats, nigerian pop",Rasaki Music Ltd / EMPIRE,ZaZa Vibes,60,2020-11-25,12
6,Mavins,128648,"afro dancehall, afropop, azontobeats, nigerian...",Mavin Records,Overloading (OVERDOSE),56,2022-05-13,1
7,Magic System,192404,"afropop, francoton, french pop, zouglou",Parlophone (France),Africainement vôtre,55,2014-05-16,15
8,Lojay,83225,"afropop, alte, nigerian pop",Metallic Music/1789,LV N ATTN,55,2021-06-04,5
9,WizKid,2409832,"afro dancehall, afropop, azonto, nigerian hip ...",Starboy/RCA Records,Essence (feat. Justin Bieber & Tems),54,2021-08-13,1


# New Section

In [ ]:
# connect to sqllite database 
conn = sqlite3.connect('spotify.db')
cursor = conn.cursor()

# afro-pop table
df_azontobeats.to_sql(name='azontobeats_table',con=conn, if_exists='replace', index=False)
conn.commit()



In [ ]:
engine = sqlalchemy.create_engine('sqlite:///spotify.db')

In [ ]:
pd.read_sql('azontobeats_table',engine)

,name,followers,song_genres,album_label,album_name,album_popularity,album_release_date,album_total_tracks
0,Kizz Daniel,609424,"afropop, azontobeats, nigerian pop",FlyBoy INC / EMPIRE,Buga (Lo Lo Lo),57,2022-05-03,1
1,L.A.X,119929,"afropop, azontobeats, nigerian pop",Rasaki Music Ltd / EMPIRE,ZaZa Vibes,60,2020-11-25,12
2,Kizz Daniel,609424,"afropop, azontobeats, nigerian pop",Flyboy I.N.C,No Bad Songz,54,2018-11-30,20
3,Wande Coal,314016,"afro dancehall, afropop, azontobeats, nigerian...",BT Music Worldwide,Iskaba,52,2016-11-25,1
4,Mavins,102882,"afro dancehall, afropop, azontobeats, nigerian...",Mavin Records,Overloading (OVERDOSE),50,2022-05-13,1
5,Adekunle Gold,690993,"afropop, alte, azontobeats, nigerian hip hop, ...",Afro Urban Records,High,50,2021-09-03,1
6,Timaya,280627,"afro dancehall, afropop, azonto, azontobeats, ...",DM Records Limited,Cold Outside,49,2021-10-14,1
7,Ric Hassani,82134,azontobeats,Riverland Records,The Prince I Became,54,2021-02-26,17
8,Mayorkun,496295,"afro dancehall, afropop, azontobeats, nigerian...",SM Entertainment West Africa Limited,Holy Father,48,2021-10-27,1
9,Maleek Berry,363048,"afro dancehall, afropop, azontobeats, nigerian...",Berry's Room,Kontrol,48,2016-05-09,1
